# Extract mturk data from sqlite

This notebook describes how I extract subject input from the participants.db that psiturk generates.

## Import required packages and setup
Before we begin our analysis, we import some required packages

In [45]:
# import packages (required for analysis)
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table
import json


# Set Parameters

## Setup experiment paths
Next, we set some useful parameters to tell pandas where our data and required files are located.

In [46]:
# params (change these for every experiment)
DATA_PATH = '/Users/kathrynschuler/Documents/current/research/raw-data/'
EXPID = '0165'
EXPTITLE = 'empiricalYang-36Noun-HFrule-easy4-mturk'


## Database parameters
Setup the parameters of the participants.db 

In [47]:
## The db_url is the location of the participants.db file on the local computer
## with 'sqlite:///' appended to the frong
## you get the particiant.db from the server location of the experiment by
## using the command :
## scp -r -P 8008 serverlogin@IP.ADD.RES.S:path/to/exp/file.db $HOME/Path/to/raw-data
db_url = "sqlite:///"+DATA_PATH+EXPID+"-"+EXPTITLE+"-data/participants.db"

## the table name gets set in config.txt of experiment file
table_name = '0165'

## data_column_name set automatically by psiturk
data_column_name = 'datastring'

# check the url to make sure it is correct
print db_url

sqlite:////Users/kathrynschuler/Documents/current/research/raw-data/0165-empiricalYang-36Noun-HFrule-easy4-mturk-data/participants.db


## Extract data into dataframe


In [48]:
# boilerplace sqlalchemy setup (don't change any of this)
engine = create_engine(db_url)
metadata = MetaData()
metadata.bind = engine
table = Table(table_name, metadata, autoload=True)

In [49]:
# make a query and loop through
s = table.select()
rows = s.execute()
# setup an empty list to hold data

data = []
#status codes of subjects who completed experiment
statuses = [3, 4, 5, 7]
# if you have workers you wish to exclude, add them here
exclude = ['A35YZM09QR7IZG:31T4R4OBOSH1FCCIYPT45L0HETLC7K']
for row in rows:
    # only use subjects who completed experiment and aren't excluded
    if row['status'] in statuses and row['uniqueid'] not in exclude:
        data.append(row[data_column_name])

In [30]:
# Now we have all participant datastrings in a list.
# Let's make it a bit easier to work with:

# parse each participant's datastring as json object
# and take the 'data' sub-object
data = [json.loads(part)['data'] for part in data]

# insert uniqueid field into trialdata in case it wasn't added
# in experiment:
for part in data:
    for record in part:
        record['trialdata']['uniqueid'] = record['uniqueid']

# flatten nested list so we just have a list of the trialdata recorded
# each time psiturk.recordTrialData(trialdata) was called.
data = [record['trialdata'] for part in data for record in part]

# Put all subjects' trial data into a dataframe object from the
# 'pandas' python library: one option among many for analysis
data_frame = pd.DataFrame(data)



## Inspect the data frame

In [35]:
# inspect the dataframe to make sure it loaded like you thought
print data_frame.head()

               action case condition correct counterbalance  det detnum error  \
0               Begin  NaN       NaN     NaN            NaN  NaN    NaN   NaN   
1            NextPage  NaN       NaN     NaN            NaN  NaN    NaN   NaN   
2            NextPage  NaN       NaN     NaN            NaN  NaN    NaN   NaN   
3            NextPage  NaN       NaN     NaN            NaN  NaN    NaN   NaN   
4  FinishInstructions  NaN       NaN     NaN            NaN  NaN    NaN   NaN   

   indexOf   n    ...                                             templates  \
0      NaN NaN    ...     [instructions/instruct-1.html, instructions/in...   
1      0.0 NaN    ...                                                   NaN   
2      1.0 NaN    ...                                                   NaN   
3      2.0 NaN    ...                                                   NaN   
4      3.0 NaN    ...                                                   NaN   

  test_number type type-full          

## Isolate task sections
Here I make a data frame for each part of the task (easier to work with)

In [36]:

# isolate the instruction sections (and only keep the variables you want)
data_instructions = data_frame[data_frame.phase == 'INSTRUCTIONS']
data_instructions = data_instructions[['action', 'phase', 'template', 'uniqueid', 'viewTime']]

# isolate the exposure phase (and only keep the variables you want)
data_production = data_frame[data_frame.phase == 'production']
data_production = data_production[['phase', 'uniqueid', 'condition', 'counterbalance', 'n', 'verb', 'noun', 'det', 'test_number', 'type','user_input']]
data_production = data_production.dropna()

# isolate the rating phase (and only keep the variables you want)
data_rating = data_frame[data_frame.phase == 'rating']
data_rating = data_rating[['phase', 'uniqueid', 'condition', 'counterbalance','n', 'verb', 'noun', 'det', 'number', 'type', 'error','user_input']]
data_rating = data_rating.dropna()



## Helpful additional things
You can inspect each one if you want to make sure they look OK.

In [38]:
print data_rating.head()

      phase                                       uniqueid condition  \
214  rating  A2541C8MY0BYV3:39ZSFO5CA8XQL1UKLP312HRZXIRUJX         1   
215  rating  A2541C8MY0BYV3:39ZSFO5CA8XQL1UKLP312HRZXIRUJX         1   
216  rating  A2541C8MY0BYV3:39ZSFO5CA8XQL1UKLP312HRZXIRUJX         1   
217  rating  A2541C8MY0BYV3:39ZSFO5CA8XQL1UKLP312HRZXIRUJX         1   
218  rating  A2541C8MY0BYV3:39ZSFO5CA8XQL1UKLP312HRZXIRUJX         1   

    counterbalance    n    verb        noun det number       type error  \
214              1  1.0  gentif     fumpogu  ka      5  incorrect    E3   
215              1  2.0  gentif     rungmot  ka      3    correct     R   
216              1  3.0  gentif  blergenfol  ka      5  incorrect   E14   
217              1  4.0  gentif    pernisel  ka      5  incorrect    E9   
218              1  5.0  gentif      melanu  ka      5  incorrect    E1   

    user_input  
214          4  
215          4  
216          4  
217          3  
218          5  


### Count how many trials people are doing
Here I am making sure everyone is doing the number of trials I think they should.

In [40]:
#make sure every subject does all 18 of the ratings.
data_rating.uniqueid.value_counts()

A3CIUPLZ6614U2:34S6N1K2ZVKRB5C0OB8FZHF197GLHT    36
A1A3YOE4YCYM4E:34X6J5FLPTZYT8N34GJWSGUAV42JQD    36
A2DWPP1KKAY0HG:3XLBSAQ9Z4DEN5NSQ3DCGJ159CMZ7F    36
A1O81SOPKHK5RZ:3WSELTNVR330KVCRRE0E4M2EQT5TAU    36
A3MEXVF979S444:3C2NJ6JBKAIDKFC3H3LNP5DONBDN2C    36
A2RVEG53L48BAE:3A0EX8ZRN8P1KRGD7YN97EEZN8VYBW    36
A1XUZFDVKP95VC:35H6S234SA1XCRPY0514L1UEDYS565    36
A1219SY7CR1UWP:3YW4XOSQKQMJQKEGV0XVYHVD487U1O    36
AGDFBU9CK6Z9R:3ATTHHXXWAPM7U498CTOANB3GSLIXJ     36
A1YM0IWPC72J7V:3IQ1VMJRYTLH0G3DT8VXDM12WHQ9AO    36
A289D98Z4GAZ28:3WYP994K17SVEFZIB761WYS79BQ6YX    36
AU849EHZNGV2Z:3HHRAGRYX8662KXE3R0FEHL7LYTO91     36
AAM0Q3DCSSQ0A:3JZQSN0I3QBRBH7TIUNZJ4YGXAXGFY     36
A36470UBRH28GO:3II4UPYCOJ8LQMNBUKXLPIZXQFEQD5    36
A32QJF67JMJFKX:3MD9PLUKKIF3QQFF6AU1CGNV8KBZNV    36
AK8ONB60Q4RJ5:3GGAI1SQEVZK0FJ52VVTO3XAUCFMCJ     36
A6HR4PTSCMZ4L:3LBXNTKX0RWTWTB64INBXUC0W2DX9T     36
A1A3TGZ7DKJWRW:3GGAI1SQEVZK0FJ52VVTO3XAWJHMC1    36
A1CE2XPYCDRHVZ:358UUM7WRZ45LE11GMJZUGZ61VPR71    36
A1K75ALN5NQI

## Extract subject input


In [41]:
# get what noun and determiner they said (input column)
print data_production['user_input'].head()

# this splits it into individual words
splits = data_production['user_input'].str.split()
data_production['det_prod'] = splits.str[2]
data_production['noun-prod'] = splits.str[1]




200     gentif daggin ka
201    gentif bleggin ka
202        gentif sep mo
203       gentif norg ka
204       gentif norg pa
Name: user_input, dtype: object


## Save to CSV

In [50]:
# for example:
data_rating.to_csv("/Users/kathrynschuler/Desktop/rating.csv")
data_production.to_csv("/Users/kathrynschuler/Desktop/production.csv")